In [25]:
import numpy as np
import pandas as pd
import math

pd.set_option('display.max_columns', None)

### Which time frame?

In [26]:
#CHANGE DEPENDING ON THE TIME FRAME IN ANALYSIS

##########################################
############ VERY IMPORTANT ##############

# Time in days (ex 24h -> 1, 48h -> 2)
Time=2

############ VERY IMPORTANT ##############
##########################################

In [27]:
Data = pd.read_csv("Data/Pneumonia_Data_48h/24h-48h/24h-48h_Data.csv")

In [28]:
Data.head()

,Key,SourceKey,Idioma,HospitalRegiaoId,HospitalCodigo,ServicoId,DoenteId,DoenteDataNascimento,DoenteIdadeEmAnos,DoenteDecadas,DoenteIdadeEmMeses,DoenteIdadeEmDias,DoenteSexoId,DistritoId,ConcelhoId,FreguesiaId,NumeroEpisodioHospitalar,DataAdmissaoUnidade,DataAdmissaoHospitalar,CamaId,TipoProvenienciaId,HospitalProvenienciaId,UnidadeProvenienciaId,GravidadeDoenteId,TipoAdmissaoId,TempoInternamentoEmDias,DataAltaUnidade,DataAltaHospitalar,HospitalDestinoId,UnidadeDestinoId,TipoDestinoAltaId,EstadoAltaId,Falecido,MediaPesoEmKg,DNR,APACHEScore,APACHEProbMorteScore,APACHECompleto,SAPSScore,SAPSProbMorteScore,SAPSCompleto,PrimeiroSOFAScore,PrimeiroSOFAHemodinamico,PrimeiroSOFAPulmonar,PrimeiroSOFARenal,PrimeiroSOFANeurologico,PrimeiroSOFAHematologico,PrimeiroSOFAHepatico,UltimoSOFAScore,UltimoSOFAHemodinamico,UltimoSOFAPulmonar,UltimoSOFARenal,UltimoSOFANeurologico,UltimoSOFAHematologico,UltimoSOFAHepatico,RankinScore,GlasgowPiorResultado,VentilacaoNaoInvasiva,IntervaloAteVentilacaoNaoInvasiva,NumeroDiasVentilacaoNaoInvasiva,NumeroPrescricoesVentilacaoNaoInvasiva,VentilacaoInvasiva,IntervaloAteVentilacaoInvasiva_DataAdmissaoHospitalar,IntervaloAteEntubacaoOrotraqueal,NumeroDiasEntubacao,NumeroDiasVentilacaoInvasiva,NumeroPrescricoesVentilacaoInvasiva,ECMO,IntervaloAteECMO,NumeroDiasECMO,NumeroPrescricoesECMO,ONAF,IntervaloAteONAF,NumeroDiasONAF,NumeroPrescricoesONAF,CPAP,CPAP_ValvulaBoussignac,PosicaoProne,Traqueostomia,TecnicaSubstituicaoRenal,TVP_TEP,Corticosteroide,NrPrescricoesCorticosteroide,Antimalarico,NrPrescricoesAntimalarico,Antivirico,NrPrescricoesAntivirico,Anticoagulante,NrPrescricoesAnticoagulante,Vasopressor,NrPrescricoesVasopressor,Broncodilatador,NrPrescricoesBroncodilatador,Psicofarmaco,NrPrescricoesPsicofarmaco,Opiaceo,NrPrescricoesOpiaceo,RelaxanteMuscular,NrPrescricoesRelaxanteMuscular,Antibiotico,NrPrescricoesAntibiotico,LineageKey,FC,TAS,TAD,TAM,Temperatura,FR,SpO2,FiO2,Lactacto,PaCO2,PaO2,pH,SaO2,Bicarbonato,metHb,BE,FiO2_Gasimetria,25-vitamina D,Ácido fólico,Ácido úrico,Albumina,ALT/TGP,Amilase,Amónia,APTT,AST/TGO,Bilirrubina direta,Bilirrubina total,BNP,Cálcio corrigido,Cálcio ionizado,Cálcio total,CK,CK-MB,CK-MB massa,Cloro,Creatinina,CTFF,D-Dímeros,DHL,Ferritina,Ferro,Fibrinogénio,Fosfatase alcalina,Fósforo,GGT,Glicose,Hemoglobina,I.N.R,Leucócitos,Linfócitos,Lipase,Magnésio,Mioglobina,Neutrófilos,PCR,Plaquetas,Potássio,Procalcitonina,Proteinas totais,Sódio,Tempo de Protrombina,Transferrina,Troponina I,Troponina T,Ureia,GlasgowScore,VIH,Hemorragia Digestiva,Patologia Cerebrovascular Isquémica,Dependência / Intoxicação,Patologia Vascular,TVP/TEP,Pneumotórax,Abdómen Agudo,Patología Cerebrovascular,Coma,Alterações Ácido-base,Encefalopatia,Anemia,Disfunção Multiorgânica,Polineuropatia / Miopatia,Traqueobronquite,Obesidade,ITU,Patologia Cerebrovascular Hemorrágica,Coagulopatia,Sepsis,Nefropatia / IRC,Alcoolismo,Tabagismo,Dislipidemia,Arritmias Cardíacas,Diabetes Mellitus,Cardiopatia não Isquémica,Choque,Insuficiência Renal Aguda,Hipertensão Arterial
0,1,3005,PT,3,15,3,2383,1951-09-30 00:00:00,66,7,800,24322,1,0.0,0.0,0.0,18007306,2018-04-04 15:20:00,2018-04-02 15:08:00,25,3,90,22.0,3,5.0,29.19444,2018-05-03 20:00:00,2018-07-11 20:40:00,90,22,3,1.0,0.0,75.0,False,32.0,76.04,True,73.0,87.09,True,11.0,4.0,2.0,1.0,4.0,0.0,0.0,5.0,0.0,2.0,0.0,3.0,0.0,0.0,-1,3,1,0.03264,0.51666,2,1,2.05486,0.06944,28.88333,25.64028,2,0,NaN,0.0,0,0,NaN,0.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,1,6,0,0,1,2,0,0,1,8,1,74.0,151.0,71.0,98.0,37.5,20.0,96.0,25.0,0.8,38.0,78.0,7.520,98.2,30.8,1.3,7.6,25.2,NaN,NaN,NaN,3.0,6.0,41.0,NaN,26.2,11.0,NaN,0.42,NaN,9.6,NaN,8.8,NaN,NaN,NaN,110.0,1.05,NaN,NaN,198.0,NaN,NaN,388.0,88.0,1.9,85.0,132.0,10.4,1.3,6.3,11.5,NaN,2.1,NaN,79.6,3.26,183.0,3.22,NaN,NaN,149.0,NaN,NaN,NaN,NaN,76.0,3.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1
1,2,3006,PT,3,15,3,2384,1946-10-23 00:00:00,71,8,858,26100,1,0.0,0.0,0.0,18007483,2018-04-04 17:56:00,2018-03-28 13:53:00,28,3,206,27.0,3,5.0,4.08611,2018-04-08 2

### Eliminate rows were the patient already was discharged of ICU (because of recovery or death)

In [29]:
from datetime import datetime
from datetime import timedelta

In [30]:
# DataAdmissaoUnidade
DataAdmissaoUnidade_str_list = Data['DataAdmissaoUnidade'].to_numpy()
DataAdmissaoUnidade_date_list =[]

for date_str in DataAdmissaoUnidade_str_list:
        
        date_date = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")        
        DataAdmissaoUnidade_date_list.append(date_date)

# DataAltaUnidade
DataAltaUnidade_str_list = Data['DataAltaUnidade'].to_numpy()
DataAltaUnidade_date_list =[]

for date_str in DataAltaUnidade_str_list:
        
        date_date = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")        
        DataAltaUnidade_date_list.append(date_date)

# Get differences between the above
Data_diference_date_list = []
for n in range(0,len(DataAdmissaoUnidade_str_list)):
        
        date_diference = DataAltaUnidade_date_list[n] - DataAdmissaoUnidade_date_list[n]
        Data_diference_date_list.append(date_diference)

# transform into days and float 
Data_float_list = []
for date_date in Data_diference_date_list:

        date_float_sec = timedelta.total_seconds(date_date)
        # transform into days
        date_float_day = date_float_sec / 86400
        Data_float_list.append(date_float_day)

In [31]:
len(Data)

15355

In [32]:
# Eliminate the rows
Data['DataDif_AdmissaoAlta'] = Data_float_list

Data = Data[Data['DataDif_AdmissaoAlta'] > Time]

Data = Data.drop(['DataDif_AdmissaoAlta'], axis=1)

In [33]:
len(Data)

12406

In [34]:
Data.head()

,Key,SourceKey,Idioma,HospitalRegiaoId,HospitalCodigo,ServicoId,DoenteId,DoenteDataNascimento,DoenteIdadeEmAnos,DoenteDecadas,DoenteIdadeEmMeses,DoenteIdadeEmDias,DoenteSexoId,DistritoId,ConcelhoId,FreguesiaId,NumeroEpisodioHospitalar,DataAdmissaoUnidade,DataAdmissaoHospitalar,CamaId,TipoProvenienciaId,HospitalProvenienciaId,UnidadeProvenienciaId,GravidadeDoenteId,TipoAdmissaoId,TempoInternamentoEmDias,DataAltaUnidade,DataAltaHospitalar,HospitalDestinoId,UnidadeDestinoId,TipoDestinoAltaId,EstadoAltaId,Falecido,MediaPesoEmKg,DNR,APACHEScore,APACHEProbMorteScore,APACHECompleto,SAPSScore,SAPSProbMorteScore,SAPSCompleto,PrimeiroSOFAScore,PrimeiroSOFAHemodinamico,PrimeiroSOFAPulmonar,PrimeiroSOFARenal,PrimeiroSOFANeurologico,PrimeiroSOFAHematologico,PrimeiroSOFAHepatico,UltimoSOFAScore,UltimoSOFAHemodinamico,UltimoSOFAPulmonar,UltimoSOFARenal,UltimoSOFANeurologico,UltimoSOFAHematologico,UltimoSOFAHepatico,RankinScore,GlasgowPiorResultado,VentilacaoNaoInvasiva,IntervaloAteVentilacaoNaoInvasiva,NumeroDiasVentilacaoNaoInvasiva,NumeroPrescricoesVentilacaoNaoInvasiva,VentilacaoInvasiva,IntervaloAteVentilacaoInvasiva_DataAdmissaoHospitalar,IntervaloAteEntubacaoOrotraqueal,NumeroDiasEntubacao,NumeroDiasVentilacaoInvasiva,NumeroPrescricoesVentilacaoInvasiva,ECMO,IntervaloAteECMO,NumeroDiasECMO,NumeroPrescricoesECMO,ONAF,IntervaloAteONAF,NumeroDiasONAF,NumeroPrescricoesONAF,CPAP,CPAP_ValvulaBoussignac,PosicaoProne,Traqueostomia,TecnicaSubstituicaoRenal,TVP_TEP,Corticosteroide,NrPrescricoesCorticosteroide,Antimalarico,NrPrescricoesAntimalarico,Antivirico,NrPrescricoesAntivirico,Anticoagulante,NrPrescricoesAnticoagulante,Vasopressor,NrPrescricoesVasopressor,Broncodilatador,NrPrescricoesBroncodilatador,Psicofarmaco,NrPrescricoesPsicofarmaco,Opiaceo,NrPrescricoesOpiaceo,RelaxanteMuscular,NrPrescricoesRelaxanteMuscular,Antibiotico,NrPrescricoesAntibiotico,LineageKey,FC,TAS,TAD,TAM,Temperatura,FR,SpO2,FiO2,Lactacto,PaCO2,PaO2,pH,SaO2,Bicarbonato,metHb,BE,FiO2_Gasimetria,25-vitamina D,Ácido fólico,Ácido úrico,Albumina,ALT/TGP,Amilase,Amónia,APTT,AST/TGO,Bilirrubina direta,Bilirrubina total,BNP,Cálcio corrigido,Cálcio ionizado,Cálcio total,CK,CK-MB,CK-MB massa,Cloro,Creatinina,CTFF,D-Dímeros,DHL,Ferritina,Ferro,Fibrinogénio,Fosfatase alcalina,Fósforo,GGT,Glicose,Hemoglobina,I.N.R,Leucócitos,Linfócitos,Lipase,Magnésio,Mioglobina,Neutrófilos,PCR,Plaquetas,Potássio,Procalcitonina,Proteinas totais,Sódio,Tempo de Protrombina,Transferrina,Troponina I,Troponina T,Ureia,GlasgowScore,VIH,Hemorragia Digestiva,Patologia Cerebrovascular Isquémica,Dependência / Intoxicação,Patologia Vascular,TVP/TEP,Pneumotórax,Abdómen Agudo,Patología Cerebrovascular,Coma,Alterações Ácido-base,Encefalopatia,Anemia,Disfunção Multiorgânica,Polineuropatia / Miopatia,Traqueobronquite,Obesidade,ITU,Patologia Cerebrovascular Hemorrágica,Coagulopatia,Sepsis,Nefropatia / IRC,Alcoolismo,Tabagismo,Dislipidemia,Arritmias Cardíacas,Diabetes Mellitus,Cardiopatia não Isquémica,Choque,Insuficiência Renal Aguda,Hipertensão Arterial
0,1,3005,PT,3,15,3,2383,1951-09-30 00:00:00,66,7,800,24322,1,0.0,0.0,0.0,18007306,2018-04-04 15:20:00,2018-04-02 15:08:00,25,3,90,22.0,3,5.0,29.19444,2018-05-03 20:00:00,2018-07-11 20:40:00,90,22,3,1.0,0.0,75.0,False,32.0,76.04,True,73.0,87.09,True,11.0,4.0,2.0,1.0,4.0,0.0,0.0,5.0,0.0,2.0,0.0,3.0,0.0,0.0,-1,3,1,0.03264,0.51666,2,1,2.05486,0.06944,28.88333,25.64028,2,0,NaN,0.0,0,0,NaN,0.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,1,6,0,0,1,2,0,0,1,8,1,74.0,151.0,71.0,98.0,37.5,20.0,96.0,25.0,0.8,38.0,78.0,7.520,98.2,30.8,1.3,7.6,25.2,NaN,NaN,NaN,3.0,6.0,41.0,NaN,26.2,11.0,NaN,0.42,NaN,9.6,NaN,8.8,NaN,NaN,NaN,110.0,1.05,NaN,NaN,198.0,NaN,NaN,388.0,88.0,1.9,85.0,132.0,10.4,1.3,6.3,11.5,NaN,2.1,NaN,79.6,3.26,183.0,3.22,NaN,NaN,149.0,NaN,NaN,NaN,NaN,76.0,3.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1
1,2,3006,PT,3,15,3,2384,1946-10-23 00:00:00,71,8,858,26100,1,0.0,0.0,0.0,18007483,2018-04-04 17:56:00,2018-03-28 13:53:00,28,3,206,27.0,3,5.0,4.08611,2018-04-08 2

### Getting some maybe usefull tables that can be used in the future

In [35]:
Admissao_Alta_Data = pd.DataFrame({'DataAdmissaoUnidade':Data['DataAdmissaoUnidade'],'DataAdmissaoHospitalar':Data['DataAdmissaoHospitalar'],'DataAltaUnidade':Data['DataAltaUnidade'],'DataAltaHospitalar':Data['DataAltaHospitalar'],'TempoInternamentoEmDias':Data['TempoInternamentoEmDias'],'DataAltaUnidade':Data['DataAltaUnidade'],'EstadoAltaId':Data['EstadoAltaId'],'Falecido':Data['Falecido'],'VentilacaoInvasiva':Data['VentilacaoInvasiva'],'IntervaloAteVentilacaoInvasiva_DataAdmissaoHospitalar':Data['IntervaloAteVentilacaoInvasiva_DataAdmissaoHospitalar'],'IntervaloAteEntubacaoOrotraqueal':Data['IntervaloAteEntubacaoOrotraqueal'],'NumeroDiasEntubacao':Data['NumeroDiasEntubacao'],'NumeroDiasVentilacaoInvasiva':Data['NumeroDiasVentilacaoInvasiva']})
Admissao_Alta_Data.head()

,DataAdmissaoUnidade,DataAdmissaoHospitalar,DataAltaUnidade,DataAltaHospitalar,TempoInternamentoEmDias,EstadoAltaId,Falecido,VentilacaoInvasiva,IntervaloAteVentilacaoInvasiva_DataAdmissaoHospitalar,IntervaloAteEntubacaoOrotraqueal,NumeroDiasEntubacao,NumeroDiasVentilacaoInvasiva
0,2018-04-04 15:20:00,2018-04-02 15:08:00,2018-05-03 20:00:00,2018-07-11 20:40:00,29.19444,1.0,0.0,1,2.05486,0.06944,28.88333,25.64028
1,2018-04-04 17:56:00,2018-03-28 13:53:00,2018-04-08 20:00:00,2018-04-08 20:00:00,4.08611,4.0,1.0,1,7.02083,NaN,4.23611,4.23403
2,2018-04-06 01:25:00,2018-04-06 01:25:00,2018-04-10 15:00:00,2018-04-16 00:00:00,4.56597,1.0,0.0,1,0.00000,0.00000,2.36389,2.36389
3,2018-04-06 02:53:00,2018-04-04 10:28:00,2018-04-11 17:00:00,2018-04-11 17:00:00,5.58819,4.0,1.0,1,1.69583,0.00764,5.58056,5.57639
4,2018-04-07 02:42:00,2018-04-06 23:47:00,2018-04-16 14:00:00,2018-05-22 14:00:00,9.47083,1.0,0.0,1,0.12153,0.00000,13.54722,9.30417


In [36]:
#SOFA have ultima versions too
Clinical_Scores = pd.DataFrame({'Falecido':Data['Falecido'],'VentilacaoInvasiva':Data['VentilacaoInvasiva'],'APACHEScore':Data['APACHEScore'],'APACHEProbMorteScore':Data['APACHEProbMorteScore'],'APACHECompleto':Data['APACHECompleto'],'SAPSScore':Data['SAPSScore'],'SAPSProbMorteScore':Data['SAPSProbMorteScore'],'SAPSCompleto':Data['SAPSCompleto'],'PrimeiroSOFAScore':Data['PrimeiroSOFAScore'],'PrimeiroSOFAHemodinamico':Data['PrimeiroSOFAHemodinamico'],'PrimeiroSOFAPulmonar':Data['PrimeiroSOFAPulmonar'],'PrimeiroSOFARenal':Data['PrimeiroSOFARenal'],'PrimeiroSOFANeurologico':Data['PrimeiroSOFANeurologico'],'PrimeiroSOFAHematologico':Data['PrimeiroSOFAHematologico'],'PrimeiroSOFAHepatico':Data['PrimeiroSOFAHepatico'],'UltimoSOFAScore':Data['UltimoSOFAScore']})
Clinical_Scores.head() 

,Falecido,VentilacaoInvasiva,APACHEScore,APACHEProbMorteScore,APACHECompleto,SAPSScore,SAPSProbMorteScore,SAPSCompleto,PrimeiroSOFAScore,PrimeiroSOFAHemodinamico,PrimeiroSOFAPulmonar,PrimeiroSOFARenal,PrimeiroSOFANeurologico,PrimeiroSOFAHematologico,PrimeiroSOFAHepatico,UltimoSOFAScore
0,0.0,1,32.0,76.04,True,73.0,87.09,True,11.0,4.0,2.0,1.0,4.0,0.0,0.0,5.0
1,1.0,1,31.0,73.28,True,70.0,83.84,True,13.0,3.0,3.0,1.0,4.0,2.0,0.0,16.0
2,0.0,1,26.0,56.93,True,61.0,70.01,True,5.0,3.0,2.0,0.0,0.0,0.0,0.0,5.0
3,1.0,1,15.0,20.97,True,44.0,32.64,True,8.0,1.0,2.0,1.0,0.0,2.0,2.0,18.0
4,0.0,1,24.0,49.68,True,63.0,73.63,True,6.0,1.0,1.0,1.0,3.0,0.0,0.0,7.0


In [37]:
# This table is more of a bachukp of ventilation related information

Ventilacao_relacionado = pd.DataFrame({'IntervaloAteEntubacaoOrotraqueal':Data['IntervaloAteEntubacaoOrotraqueal'],'NumeroDiasEntubacao':Data['NumeroDiasEntubacao'],'CPAP':Data['CPAP'],'CPAP_ValvulaBoussignac':Data['CPAP_ValvulaBoussignac'],'ONAF':Data['ONAF'],'IntervaloAteONAF':Data['IntervaloAteONAF'],'NumeroDiasEntubacao':Data['NumeroDiasEntubacao'],'CPAP':Data['CPAP'],'CPAP_ValvulaBoussignac':Data['CPAP_ValvulaBoussignac'],'ONAF':Data['ONAF'],'IntervaloAteONAF':Data['IntervaloAteONAF'],'NumeroDiasEntubacao':Data['NumeroDiasEntubacao'],'CPAP':Data['CPAP'],'CPAP_ValvulaBoussignac':Data['CPAP_ValvulaBoussignac'],'ONAF':Data['ONAF'],'IntervaloAteONAF':Data['IntervaloAteONAF'],'NumeroDiasONAF':Data['NumeroDiasONAF'],'VentilacaoNaoInvasiva':Data['VentilacaoNaoInvasiva'],'IntervaloAteVentilacaoNaoInvasiva':Data['IntervaloAteVentilacaoNaoInvasiva'],'NumeroDiasVentilacaoNaoInvasiva':Data['NumeroDiasVentilacaoNaoInvasiva'],'NumeroDiasVentilacaoNaoInvasiva':Data['NumeroDiasVentilacaoNaoInvasiva'],'IntervaloAteVentilacaoInvasiva_DataAdmissaoHospitalar':Data['IntervaloAteVentilacaoInvasiva_DataAdmissaoHospitalar'],'NumeroDiasVentilacaoInvasiva':Data['NumeroDiasVentilacaoInvasiva']})
Ventilacao_relacionado.head() 

,IntervaloAteEntubacaoOrotraqueal,NumeroDiasEntubacao,CPAP,CPAP_ValvulaBoussignac,ONAF,IntervaloAteONAF,NumeroDiasONAF,VentilacaoNaoInvasiva,IntervaloAteVentilacaoNaoInvasiva,NumeroDiasVentilacaoNaoInvasiva,IntervaloAteVentilacaoInvasiva_DataAdmissaoHospitalar,NumeroDiasVentilacaoInvasiva
0,0.06944,28.88333,0,0,0,NaN,0.0,1,0.03264,0.51666,2.05486,25.64028
1,NaN,4.23611,0,0,0,NaN,0.0,0,NaN,0.00000,7.02083,4.23403
2,0.00000,2.36389,0,0,0,NaN,0.0,1,2.50972,1.88681,0.00000,2.36389
3,0.00764,5.58056,0,0,0,NaN,0.0,0,NaN,0.00000,1.69583,5.57639
4,0.00000,13.54722,0,0,0,NaN,0.0,0,NaN,0.00000,0.12153,9.30417


### Dropping data that isn't usefull for prediction

In [38]:
# in age, I can use DoenteIdadeEmMeses, DoenteIdadeEmDias or DoenteIdadeEmAnos	
Data = Data.drop(['Key','SourceKey','Idioma','HospitalRegiaoId','HospitalCodigo','ServicoId','DoenteId','DoenteDataNascimento','DoenteDecadas','DoenteIdadeEmMeses','DoenteIdadeEmDias','DistritoId','ConcelhoId','FreguesiaId','NumeroEpisodioHospitalar','DataAdmissaoUnidade','DataAdmissaoHospitalar','CamaId','TipoProvenienciaId','HospitalProvenienciaId','UnidadeProvenienciaId','TipoAdmissaoId','TempoInternamentoEmDias','DataAltaUnidade','DataAltaHospitalar','HospitalDestinoId','HospitalDestinoId','UnidadeDestinoId','TipoDestinoAltaId','EstadoAltaId','DNR','APACHEScore','APACHEProbMorteScore','APACHECompleto','SAPSScore','SAPSProbMorteScore','SAPSCompleto','PrimeiroSOFAScore','PrimeiroSOFAHemodinamico','PrimeiroSOFAPulmonar','PrimeiroSOFARenal','PrimeiroSOFANeurologico','PrimeiroSOFAHematologico','PrimeiroSOFAHepatico','UltimoSOFAScore','UltimoSOFAHemodinamico','UltimoSOFAPulmonar','UltimoSOFARenal','UltimoSOFANeurologico','UltimoSOFAHematologico','UltimoSOFAHepatico','RankinScore','GlasgowPiorResultado','NumeroPrescricoesVentilacaoNaoInvasiva','NumeroPrescricoesVentilacaoInvasiva','NumeroPrescricoesECMO','NumeroPrescricoesONAF','IntervaloAteEntubacaoOrotraqueal','NumeroDiasEntubacao','ECMO','IntervaloAteECMO','NumeroDiasECMO','CPAP','CPAP_ValvulaBoussignac','PosicaoProne','Traqueostomia','TecnicaSubstituicaoRenal','TVP_TEP','Corticosteroide','NrPrescricoesCorticosteroide','Antimalarico','NrPrescricoesAntimalarico','Antivirico','NrPrescricoesAntivirico','Anticoagulante','NrPrescricoesAnticoagulante','Vasopressor','NrPrescricoesVasopressor','Broncodilatador','NrPrescricoesBroncodilatador','Psicofarmaco','NrPrescricoesPsicofarmaco','Opiaceo','NrPrescricoesOpiaceo','RelaxanteMuscular','NrPrescricoesRelaxanteMuscular','Antibiotico','NrPrescricoesAntibiotico','LineageKey','GravidadeDoenteId','ONAF','IntervaloAteONAF','NumeroDiasONAF'], axis=1)

In [39]:
# Here we turn dates from the non invasive and invase ventilation into data that can be directly used for prediction

IntervaloAteVNI = Data['IntervaloAteVentilacaoNaoInvasiva'].to_numpy()
VentilacaoNaoInvasivaAteData =[]

for t in IntervaloAteVNI:
    if t<=Time:
        VentilacaoNaoInvasivaAteData.append(1)
    else:
        VentilacaoNaoInvasivaAteData.append(0)

IntervaloAteVI = Data['IntervaloAteVentilacaoInvasiva_DataAdmissaoHospitalar'].to_numpy()
VentilacaoInvasivaAteData =[]

for t in IntervaloAteVI:
    if t<=Time:
        VentilacaoInvasivaAteData.append(1)
    else:
        VentilacaoInvasivaAteData.append(0)
    

In [40]:
IntervaloAteVNI

array([0.03264,     nan, 2.50972, ..., 4.98472,     nan, 6.87639])

In [41]:
Data['VentilacaoNaoInvasivaAteData'] = VentilacaoNaoInvasivaAteData
Data['VentilacaoInvasivaAteData'] = VentilacaoInvasivaAteData

In [42]:
Data = Data.drop(['VentilacaoNaoInvasiva','IntervaloAteVentilacaoNaoInvasiva','NumeroDiasVentilacaoNaoInvasiva','IntervaloAteVentilacaoInvasiva_DataAdmissaoHospitalar','NumeroDiasVentilacaoInvasiva'], axis=1)

In [43]:
Data.head()

,DoenteIdadeEmAnos,DoenteSexoId,Falecido,MediaPesoEmKg,VentilacaoInvasiva,FC,TAS,TAD,TAM,Temperatura,FR,SpO2,FiO2,Lactacto,PaCO2,PaO2,pH,SaO2,Bicarbonato,metHb,BE,FiO2_Gasimetria,25-vitamina D,Ácido fólico,Ácido úrico,Albumina,ALT/TGP,Amilase,Amónia,APTT,AST/TGO,Bilirrubina direta,Bilirrubina total,BNP,Cálcio corrigido,Cálcio ionizado,Cálcio total,CK,CK-MB,CK-MB massa,Cloro,Creatinina,CTFF,D-Dímeros,DHL,Ferritina,Ferro,Fibrinogénio,Fosfatase alcalina,Fósforo,GGT,Glicose,Hemoglobina,I.N.R,Leucócitos,Linfócitos,Lipase,Magnésio,Mioglobina,Neutrófilos,PCR,Plaquetas,Potássio,Procalcitonina,Proteinas totais,Sódio,Tempo de Protrombina,Transferrina,Troponina I,Troponina T,Ureia,GlasgowScore,VIH,Hemorragia Digestiva,Patologia Cerebrovascular Isquémica,Dependência / Intoxicação,Patologia Vascular,TVP/TEP,Pneumotórax,Abdómen Agudo,Patología Cerebrovascular,Coma,Alterações Ácido-base,Encefalopatia,Anemia,Disfunção Multiorgânica,Polineuropatia / Miopatia,Traqueobronquite,Obesidade,ITU,Patologia Cerebrovascular Hemorrágica,Coagulopatia,Sepsis,Nefropatia / IRC,Alcoolismo,Tabagismo,Dislipidemia,Arritmias Cardíacas,Diabetes Mellitus,Cardiopatia não Isquémica,Choque,Insuficiência Renal Aguda,Hipertensão Arterial,VentilacaoNaoInvasivaAteData,VentilacaoInvasivaAteData
0,66,1,0.0,75.0,1,74.0,151.0,71.0,98.0,37.5,20.0,96.0,25.0,0.8,38.0,78.0,7.520,98.2,30.8,1.3,7.6,25.2,NaN,NaN,NaN,3.0,6.0,41.0,NaN,26.2,11.0,NaN,0.42,NaN,9.6,NaN,8.8,NaN,NaN,NaN,110.0,1.05,NaN,NaN,198.0,NaN,NaN,388.0,88.0,1.9,85.0,132.0,10.4,1.3,6.3,11.5,NaN,2.1,NaN,79.6,3.26,183.0,3.22,NaN,NaN,149.0,NaN,NaN,NaN,NaN,76.0,3.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0
1,71,1,1.0,85.0,1,90.0,102.0,56.0,71.0,36.1,27.0,96.0,90.0,2.5,39.0,62.0,7.320,96.2,20.5,0.1,-5.6,100.0,NaN,NaN,NaN,2.0,430.0,NaN,NaN,NaN,1858.0,NaN,2.12,NaN,9.0,NaN,7.4,1369.0,NaN,NaN,100.0,2.43,NaN,NaN,2386.0,NaN,NaN,NaN,190.0,4.4,436.0,NaN,12.7,1.1,3.0,3.8,141.0,2.2,NaN,93.4,8.73,82.0,4.86,NaN,NaN,135.0,NaN,NaN,NaN,NaN,121.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0
2,54,1,0.0,47.0,1,78.0,140.0,78.0,99.0,36.9,14.0,91.0,50.0,2.4,43.0,102.0,7.490,99.6,32.8,0.9,9.5,NaN,NaN,NaN,NaN,2.4,13.0,NaN,NaN,28.8,12.0,NaN,0.34,NaN,9.4,NaN,8.1,NaN,NaN,NaN,101.0,0.29,NaN,NaN,242.0,NaN,NaN,863.0,64.0,2.6,27.0,205.0,9.6,1.1,7.1,4.2,NaN,1.9,NaN,89.8,16.08,152.0,3.16,NaN,NaN,142.0,NaN,NaN,NaN,NaN,19.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1
3,61,1,1.0,70.0,1,92.0,116.0,47.0,69.0,37.3,15.0,95.0,NaN,1.5,31.8,64.1,7.396,92.5,19.8,NaN,NaN,21.0,NaN,NaN,NaN,2.9,17.0,NaN,NaN,44.9,36.0,1.51,2.60,NaN,9.9,NaN,9.0,NaN,NaN,NaN,109.0,1.23,NaN,NaN,360.0,NaN,NaN,NaN,NaN,4.6,35.0,70.0,8.5,1.6,24.2,6.6,NaN,1.9,NaN,89.0,2.49,77.0,3.77,NaN,NaN,145.0,NaN,NaN,NaN,NaN,134.0,15.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
4,73,1,0.0,80.0,1,44.0,165.0,58.0,93.0,36.0,10.0,100.0,35.0,1.5,40.0,100.0,7.420,98.3,25.9,0.8,1.3,40.0,NaN,NaN,NaN,3.3,NaN,NaN,NaN,NaN,22.0,NaN,0.47,NaN,8.8,NaN,8.2,NaN,NaN,NaN,111.0,1.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,164.0,11.8,1.2,11.6,6.9,NaN,2.2,NaN,86.8,20.99,155.0,3.93,NaN,NaN,145.0,NaN,NaN,NaN,NaN,55.0,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,1


### Eliminating Antecedentes with low amount

In [44]:
Antecedentes_Count = Data.sum()
Antecedentes_Count = Antecedentes_Count.to_frame()
Antecedentes_Count['feature'] = Antecedentes_Count.index
Antecedentes_Count = Antecedentes_Count.iloc[72::]
Antecedentes_Count.head()

,0,feature
VIH,168.0,VIH
Hemorragia Digestiva,194.0,Hemorragia Digestiva
Patologia Cerebrovascular Isquémica,215.0,Patologia Cerebrovascular Isquémica
Dependência / Intoxicação,249.0,Dependência / Intoxicação
Patologia Vascular,266.0,Patologia Vascular


In [45]:
ToDrop = ['VIH','Hemorragia Digestiva','Patologia Cerebrovascular Isquémica']

Data = Data.drop(ToDrop, axis=1)

###  Putting prediction columns in the end

In [46]:
# Puting the prediction columns in the end 

cols = Data.columns.tolist()
cols.remove('Falecido')
cols.remove('VentilacaoInvasiva')
cols.append('Falecido')
cols.append('VentilacaoInvasiva')

Data = Data[cols]

In [47]:
# There is already another FiO2 that has slitly less nans
del Data['FiO2_Gasimetria']

### Changing column names

In [48]:
Data.head()

,DoenteIdadeEmAnos,DoenteSexoId,MediaPesoEmKg,FC,TAS,TAD,TAM,Temperatura,FR,SpO2,FiO2,Lactacto,PaCO2,PaO2,pH,SaO2,Bicarbonato,metHb,BE,25-vitamina D,Ácido fólico,Ácido úrico,Albumina,ALT/TGP,Amilase,Amónia,APTT,AST/TGO,Bilirrubina direta,Bilirrubina total,BNP,Cálcio corrigido,Cálcio ionizado,Cálcio total,CK,CK-MB,CK-MB massa,Cloro,Creatinina,CTFF,D-Dímeros,DHL,Ferritina,Ferro,Fibrinogénio,Fosfatase alcalina,Fósforo,GGT,Glicose,Hemoglobina,I.N.R,Leucócitos,Linfócitos,Lipase,Magnésio,Mioglobina,Neutrófilos,PCR,Plaquetas,Potássio,Procalcitonina,Proteinas totais,Sódio,Tempo de Protrombina,Transferrina,Troponina I,Troponina T,Ureia,GlasgowScore,Dependência / Intoxicação,Patologia Vascular,TVP/TEP,Pneumotórax,Abdómen Agudo,Patología Cerebrovascular,Coma,Alterações Ácido-base,Encefalopatia,Anemia,Disfunção Multiorgânica,Polineuropatia / Miopatia,Traqueobronquite,Obesidade,ITU,Patologia Cerebrovascular Hemorrágica,Coagulopatia,Sepsis,Nefropatia / IRC,Alcoolismo,Tabagismo,Dislipidemia,Arritmias Cardíacas,Diabetes Mellitus,Cardiopatia não Isquémica,Choque,Insuficiência Renal Aguda,Hipertensão Arterial,VentilacaoNaoInvasivaAteData,VentilacaoInvasivaAteData,Falecido,VentilacaoInvasiva
0,66,1,75.0,74.0,151.0,71.0,98.0,37.5,20.0,96.0,25.0,0.8,38.0,78.0,7.520,98.2,30.8,1.3,7.6,NaN,NaN,NaN,3.0,6.0,41.0,NaN,26.2,11.0,NaN,0.42,NaN,9.6,NaN,8.8,NaN,NaN,NaN,110.0,1.05,NaN,NaN,198.0,NaN,NaN,388.0,88.0,1.9,85.0,132.0,10.4,1.3,6.3,11.5,NaN,2.1,NaN,79.6,3.26,183.0,3.22,NaN,NaN,149.0,NaN,NaN,NaN,NaN,76.0,3.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0.0,1
1,71,1,85.0,90.0,102.0,56.0,71.0,36.1,27.0,96.0,90.0,2.5,39.0,62.0,7.320,96.2,20.5,0.1,-5.6,NaN,NaN,NaN,2.0,430.0,NaN,NaN,NaN,1858.0,NaN,2.12,NaN,9.0,NaN,7.4,1369.0,NaN,NaN,100.0,2.43,NaN,NaN,2386.0,NaN,NaN,NaN,190.0,4.4,436.0,NaN,12.7,1.1,3.0,3.8,141.0,2.2,NaN,93.4,8.73,82.0,4.86,NaN,NaN,135.0,NaN,NaN,NaN,NaN,121.0,3.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,1.0,1
2,54,1,47.0,78.0,140.0,78.0,99.0,36.9,14.0,91.0,50.0,2.4,43.0,102.0,7.490,99.6,32.8,0.9,9.5,NaN,NaN,NaN,2.4,13.0,NaN,NaN,28.8,12.0,NaN,0.34,NaN,9.4,NaN,8.1,NaN,NaN,NaN,101.0,0.29,NaN,NaN,242.0,NaN,NaN,863.0,64.0,2.6,27.0,205.0,9.6,1.1,7.1,4.2,NaN,1.9,NaN,89.8,16.08,152.0,3.16,NaN,NaN,142.0,NaN,NaN,NaN,NaN,19.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0.0,1
3,61,1,70.0,92.0,116.0,47.0,69.0,37.3,15.0,95.0,NaN,1.5,31.8,64.1,7.396,92.5,19.8,NaN,NaN,NaN,NaN,NaN,2.9,17.0,NaN,NaN,44.9,36.0,1.51,2.60,NaN,9.9,NaN,9.0,NaN,NaN,NaN,109.0,1.23,NaN,NaN,360.0,NaN,NaN,NaN,NaN,4.6,35.0,70.0,8.5,1.6,24.2,6.6,NaN,1.9,NaN,89.0,2.49,77.0,3.77,NaN,NaN,145.0,NaN,NaN,NaN,NaN,134.0,15.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1.0,1
4,73,1,80.0,44.0,165.0,58.0,93.0,36.0,10.0,100.0,35.0,1.5,40.0,100.0,7.420,98.3,25.9,0.8,1.3,NaN,NaN,NaN,3.3,NaN,NaN,NaN,NaN,22.0,NaN,0.47,NaN,8.8,NaN,8.2,NaN,NaN,NaN,111.0,1.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,164.0,11.8,1.2,11.6,6.9,NaN,2.2,NaN,86.8,20.99,155.0,3.93,NaN,NaN,145.0,NaN,NaN,NaN,NaN,55.0,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0.0,1


In [49]:
Data.columns = ["Age", "Gender", "Weight", 'HeartRate', 'TAS', 'TAD', 'TAM', 'Temperature', 'RR', 'SpO2', 'FiO2', 'Lactate', 'PaCO2',
 'PaO2', 'pH', 'SaO2', 'Bicarbonate', 'metHb', 'BE', 'VitaminD', 'Folic', 'Uric', 'Albumin', 'ALT/TGP', 'Amylase', 'Ammonia', 'APTT',
 'AST/TGO', 'DBilirubin', 'Tbilirubin', 'BNP', 'CCalcium', 'ICalcium', 'TCalcium', 'CK', 'CK-MB', 'CK-MBWeight', 'Chlorine',
 'Creatinine', 'CTFF', 'D-Dimers', 'DHL', 'Ferritin', 'Iron', 'Fibrinogen', 'AlkalinePhosph', 'Phosphorus', 'GGT', 'Glucose',
 'Hemoglobin', 'I.N.R', 'Leukocytes', 'Lymphocytes', 'Lipase', 'Magnesium', 'Myoglobin', 'Neutrophils', 'PCR', 'Platelets',
 'Potassium', 'Procalcitonin', 'TProteins', 'Sodium', 'Prothrombin', 'Transferrin', 'TroponinI', 'TroponinT', 'Urea', 'Glasgow',
 'AddicIntoxic', 'VascularP', 'TVP/TEP', 'Pneumothorax', 'AAbdomen', 'CerebrovasculaP', 'Coma', 'AcidBaseC', 'Encephalopathy',
 'Anemia', 'MultiorganD', 'PolyneuroMyopathy', 'Tracheobronchitis', 'Obesity', 'ITU', 'HemorrhagicCerebroP', 'Coagulopathy',
 'Sepsis', 'Nephro/IRC', 'Alcoholism', 'Smoking', 'Dyslipidemia', 'Arrhythmias', 'Diabetes', 'NIHeartDisease', 'Shock', 'RenalFailure',
 'Hypertension', 'NIVUD', 'IMVUD', 'Mortality', 'IMV']

In [50]:
Data.head()

,Age,Gender,Weight,HeartRate,TAS,TAD,TAM,Temperature,RR,SpO2,FiO2,Lactate,PaCO2,PaO2,pH,SaO2,Bicarbonate,metHb,BE,VitaminD,Folic,Uric,Albumin,ALT/TGP,Amylase,Ammonia,APTT,AST/TGO,DBilirubin,Tbilirubin,BNP,CCalcium,ICalcium,TCalcium,CK,CK-MB,CK-MBWeight,Chlorine,Creatinine,CTFF,D-Dimers,DHL,Ferritin,Iron,Fibrinogen,AlkalinePhosph,Phosphorus,GGT,Glucose,Hemoglobin,I.N.R,Leukocytes,Lymphocytes,Lipase,Magnesium,Myoglobin,Neutrophils,PCR,Platelets,Potassium,Procalcitonin,TProteins,Sodium,Prothrombin,Transferrin,TroponinI,TroponinT,Urea,Glasgow,AddicIntoxic,VascularP,TVP/TEP,Pneumothorax,AAbdomen,CerebrovasculaP,Coma,AcidBaseC,Encephalopathy,Anemia,MultiorganD,PolyneuroMyopathy,Tracheobronchitis,Obesity,ITU,HemorrhagicCerebroP,Coagulopathy,Sepsis,Nephro/IRC,Alcoholism,Smoking,Dyslipidemia,Arrhythmias,Diabetes,NIHeartDisease,Shock,RenalFailure,Hypertension,NIVUD,IMVUD,Mortality,IMV
0,66,1,75.0,74.0,151.0,71.0,98.0,37.5,20.0,96.0,25.0,0.8,38.0,78.0,7.520,98.2,30.8,1.3,7.6,NaN,NaN,NaN,3.0,6.0,41.0,NaN,26.2,11.0,NaN,0.42,NaN,9.6,NaN,8.8,NaN,NaN,NaN,110.0,1.05,NaN,NaN,198.0,NaN,NaN,388.0,88.0,1.9,85.0,132.0,10.4,1.3,6.3,11.5,NaN,2.1,NaN,79.6,3.26,183.0,3.22,NaN,NaN,149.0,NaN,NaN,NaN,NaN,76.0,3.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0.0,1
1,71,1,85.0,90.0,102.0,56.0,71.0,36.1,27.0,96.0,90.0,2.5,39.0,62.0,7.320,96.2,20.5,0.1,-5.6,NaN,NaN,NaN,2.0,430.0,NaN,NaN,NaN,1858.0,NaN,2.12,NaN,9.0,NaN,7.4,1369.0,NaN,NaN,100.0,2.43,NaN,NaN,2386.0,NaN,NaN,NaN,190.0,4.4,436.0,NaN,12.7,1.1,3.0,3.8,141.0,2.2,NaN,93.4,8.73,82.0,4.86,NaN,NaN,135.0,NaN,NaN,NaN,NaN,121.0,3.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,1.0,1
2,54,1,47.0,78.0,140.0,78.0,99.0,36.9,14.0,91.0,50.0,2.4,43.0,102.0,7.490,99.6,32.8,0.9,9.5,NaN,NaN,NaN,2.4,13.0,NaN,NaN,28.8,12.0,NaN,0.34,NaN,9.4,NaN,8.1,NaN,NaN,NaN,101.0,0.29,NaN,NaN,242.0,NaN,NaN,863.0,64.0,2.6,27.0,205.0,9.6,1.1,7.1,4.2,NaN,1.9,NaN,89.8,16.08,152.0,3.16,NaN,NaN,142.0,NaN,NaN,NaN,NaN,19.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0.0,1
3,61,1,70.0,92.0,116.0,47.0,69.0,37.3,15.0,95.0,NaN,1.5,31.8,64.1,7.396,92.5,19.8,NaN,NaN,NaN,NaN,NaN,2.9,17.0,NaN,NaN,44.9,36.0,1.51,2.60,NaN,9.9,NaN,9.0,NaN,NaN,NaN,109.0,1.23,NaN,NaN,360.0,NaN,NaN,NaN,NaN,4.6,35.0,70.0,8.5,1.6,24.2,6.6,NaN,1.9,NaN,89.0,2.49,77.0,3.77,NaN,NaN,145.0,NaN,NaN,NaN,NaN,134.0,15.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1.0,1
4,73,1,80.0,44.0,165.0,58.0,93.0,36.0,10.0,100.0,35.0,1.5,40.0,100.0,7.420,98.3,25.9,0.8,1.3,NaN,NaN,NaN,3.3,NaN,NaN,NaN,NaN,22.0,NaN,0.47,NaN,8.8,NaN,8.2,NaN,NaN,NaN,111.0,1.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,164.0,11.8,1.2,11.6,6.9,NaN,2.2,NaN,86.8,20.99,155.0,3.93,NaN,NaN,145.0,NaN,NaN,NaN,NaN,55.0,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0.0,1


### Saving new Dataframes

In [51]:
Data.to_csv('Data/Pneumonia_Data_48h/24h-48h/24h-48h_Organized_Data.csv', index=False)

In [52]:
Admissao_Alta_Data.to_csv('Data/Other interesting Data/Admission_and_Discharge_Data.csv', index=False)
Clinical_Scores.to_csv('Data/Other interesting Data/Clinical_Scores_Data.csv', index=False)
Ventilacao_relacionado.to_csv('Data/Other interesting Data/Related_Ventilation_Data.csv', index=False)